In [54]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)

query = text("""SELECT
	fc.Suc_Id,
	fc.Emp_Id,
	fc.Caja_Id,
    fc.Factura_Id,
	fc.Factura_Fecha,
	fc.Factura_FechaHora,
	fc.AnioMes_Id,
	fc.Cliente_Id,
	fc.MonederoTarj_Id,
	fc.Vendedor_Id,
	md.Tipo_Plan,
	md.Edad,
	suc.TipoSucursal_Id,
	suc.TipoSucursal_Nombre,
	suc.Zona_Id,
	suc.Zona_Nombre,
	COUNT ( fp.Factura_Id ) AS Cantidad_Productos_Vendidos 
FROM
	[dbo].[BI_Kielsa_Hecho_FacturaEncabezado] AS fc
	LEFT JOIN [dbo].[BI_Dim_Monedero_Kielsa] AS md ON md.Monedero_Id = fc.Monedero_Id
	LEFT JOIN [dbo].[BI_Dim_Sucursal_Kielsa] AS suc ON suc.Sucursal_Id = CONCAT ( fc.Emp_Id, '-', fc.Suc_Id )
	LEFT JOIN [dbo].[BI_Kielsa_Hecho_FacturaPosicion] AS fp ON fp.Factura_Id = fc.Factura_Id 
	AND fp.Suc_Id = fc.Suc_Id 
	AND fp.Caja_Id = fc.Caja_Id 
	AND fp.Emp_Id = fc.Emp_Id 
WHERE
	fc.AnioMes_Id = N'202410' 
	AND fc.Emp_Id = 1 
GROUP BY
	fc.Suc_Id,
	fc.Emp_Id,
	fc.Caja_Id,
    fc.Factura_Id,
	fc.Factura_Fecha,
	fc.Factura_FechaHora,
	fc.AnioMes_Id,
	fc.Cliente_Id,
	fc.MonederoTarj_Id,
	fc.Vendedor_Id,
	md.Tipo_Plan,
	md.Edad,
	suc.TipoSucursal_Id,
	suc.TipoSucursal_Nombre,
	suc.Zona_Id,
	suc.Zona_Nombre;""")

with engine.connect() as conn:
    df = pd.read_sql(query, conn, index_col='Factura_Id')


In [55]:
df.head()

,Suc_Id,Emp_Id,Caja_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre,Cantidad_Productos_Vendidos
Factura_Id,,,,,,,,,,,,,,,,
90665,239,1,3,2024-10-14,14:31:06,202410,10000001,0,10004159,Monedero Farmex Plus,50,1-2,Centro Comercial,1-1,CENTRO SUR,1
136545,23,1,4,2024-10-17,18:33:52,202410,10000001,0801-1975-03829,10004051,Monedero todo publico,49,1-7,Stand Alone,1-1,CENTRO SUR,1
24541,180,1,1,2024-10-15,12:06:03,202410,10000002,0506-1947-00530,10003779,Monedero tercera edad,77,1-6,Popular,1-2,NORTE,2
118646,109,1,3,2024-10-15,17:33:12,202410,10000001,0,10003871,Monedero Farmex Plus,50,1-2,Centro Comercial,1-1,CENTRO SUR,3
117871,207,1,1,2024-10-15,20:38:40,202410,10000001,0201-1986-00137,10003745,Monedero todo publico,39,1-9,Plaza Comercial,1-2,NORTE,1


In [60]:
# valores unicos de tipo TipoSucursal_Nombre
df.TipoSucursal_Nombre.unique()


array(['Centro Comercial', 'Stand Alone', 'Popular', 'Plaza Comercial',
       'Clínica/Hospitalaria', 'Supermercado', 'Estación de Servicio',
       'General', 'Especialidades Oncologicas'], dtype=object)

In [61]:
df.Tipo_Plan.unique()

array(['Monedero Farmex  Plus', 'Monedero todo publico',
       'Monedero tercera edad', 'Monedero Cuarta Edad',
       'Monedero Kielsa Cash', 'Cronico', 'Monedero Clínica TP',
       'Club de Médicos', 'Monedero Clínica TE',
       'Monedero Kielsa Cash Tercera edad'], dtype=object)

In [58]:
# convertir los campos Factura_Fecha y Factura_FechaHora a datetime para poder trabajar con ellos
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha']) 